## News_scrape for Google news

In [1]:
## import the packages we need 
import requests
from bs4 import BeautifulSoup
import re

In [2]:
## the package to scrape the content of a news page 
from goose3 import Goose  

### Step 1 Get the links of news in Google News

In [3]:
def search(key_word, n):
    k = []
    for i in range(0,n,10): #10 stands for data till page 1, 20 stands for data till page 2
        search = 'https://www.google.com.mx/search?q=key_word&tbm=nws&ei=ThN-XvHFOr-zytMPjN2GQA&start=0'
        url=search.replace('key_word',key_word)## change the search words
        browser = 'Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US; rv:1.9.0.7) Gecko/2009021910 Firefox/3.0.7' ## shown as web users
        headers = {'User-Agent':browser,}
        new_link = re.sub('start=\d+','start=%s'%i, url, re.S)#collect links from pages we want
        page = requests.get(new_link)
        soup = BeautifulSoup(page.content, "lxml")
        links = soup.findAll("a")  ## try to get the links we want
        l = []  ## the list to store our news links
        for link in  soup.find_all("a",href=re.compile("(?<=/url\?q=)(htt.*://.*)")):
            l.append(re.split(":(?=http)",link["href"].replace("/url?q=",""))[0]) ## to decode the link for each page
        for i in l:
            k.append(i[:i.index('&sa')])
    return(set(k))


## Step 2 To scrape the content of the news

In [4]:
result = search("Jones+%26+Jones",50) #collect news from latest 2 pages for Jones&Jones
count_news = len(result) #the number of list of news crawped
print(count_news)
g = Goose()
all_context = ""
for i in result:
    try:
        article = g.extract(url=i)
        all_context = all_context +article.cleaned_text +"\r\n"
    except:
        all_context = all_contect+""

#print(all_content)

55


## Sentiment Method

In [5]:
negitive_dic = open(r"negitive_word.txt",encoding ="utf8", errors="ignore")
negitive_dic = negitive_dic.read()
negitive_word = negitive_dic.split()

FileNotFoundError: [Errno 2] No such file or directory: 'negitive_word.txt'

In [7]:
posstive_dic = open(r"posstive_word.txt",encoding ="utf8", errors="ignore")
posstive_dic = posstive_dic.read()
posstive_word = posstive_dic.split()

In [8]:
def segmentation(sentence):
    seg_list = sentence.split()
    seg_result = []
    for w in seg_list:
        seg_result.append(w)
    return seg_result

In [9]:
def del_stopWord(seg_result):
    stop_dic = open('stop_word.txt')
    stopwords = stop_dic.readlines()
    arr = []
    for stopword in stopwords:
        stopword = stopword.replace("\n", "")
        arr.append(stopword)
    new_sent = []
    for word in seg_result:
        if word not in arr:
            new_sent.append(word)
    return new_sent

In [10]:
def sentence_score(sentences):
    seg_result = segmentation(sentences)
    seg_result = del_stopWord(seg_result)
    poscount = 0
    negcount = 0

    for word in seg_result:
        if word in negitive_word:
            negcount +=1
        elif word in posstive_word:
            poscount +=1
    final_score = (poscount - negcount)/(poscount + negcount)

    print('Positive Score:',poscount)
    print('Negative Score:',negcount)
    print('Final Score:' ,final_score)

## Output Sentiment Score

In [13]:
sentences = all_context
sentence_score(sentences)

Positive Score: 426
Negative Score: 208
Final Score: 0.3438485804416404
